In [1]:
from modules.tweet_data import read_raw_data
from modules.topics import TopicSeries
#from modules.tokens import spacy_twitter_model, twitter_tokenizer

import datetime as dt
import pandas as pd
import spacy
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import matplotlib.pyplot as plt

In [2]:
tweet_df = read_raw_data()

Reading data


In [ ]:
tweet_df.head()

In [6]:
start = dt.datetime(2018, 6, 1) + dt.timedelta(hours=15) + dt.timedelta(minutes=45)
end = dt.datetime(2020, 5, 31) + dt.timedelta(hours=15) + dt.timedelta(minutes=45)
date_range = pd.date_range(start=start, end=end , freq='1D')
date_range

DatetimeIndex(['2018-06-01 14:45:00', '2018-06-02 14:45:00',
               '2018-06-03 14:45:00', '2018-06-04 14:45:00',
               '2018-06-05 14:45:00', '2018-06-06 14:45:00',
               '2018-06-07 14:45:00', '2018-06-08 14:45:00',
               '2018-06-09 14:45:00', '2018-06-10 14:45:00',
               ...
               '2020-05-22 14:45:00', '2020-05-23 14:45:00',
               '2020-05-24 14:45:00', '2020-05-25 14:45:00',
               '2020-05-26 14:45:00', '2020-05-27 14:45:00',
               '2020-05-28 14:45:00', '2020-05-29 14:45:00',
               '2020-05-30 14:45:00', '2020-05-31 14:45:00'],
              dtype='datetime64[ns]', length=731, freq='D')

In [20]:
tweet_df[date_range[0]:(date_range[1]-dt.timedelta(seconds=1))]

,tweet_id,handle,tweet
timestamp,,,
2020-05-12 16:00:01,1260238306593439747,DailyFXTeam,"Indices Update: As of 16:00, these are your be..."
2020-05-12 16:00:01,1260238308090744832,Brad_Setser,"In fact, the swap lines were so effective at b..."
2020-05-12 16:00:01,1260238308241629187,AJEnglish,This low-cost eco-friendly alternative to the ...
2020-05-12 16:00:01,1260238308887666688,HouseFloor,"12 noon, Tuesday, May 12, 2020 Program for Tu..."
2020-05-12 16:00:01,1260238309177073670,RT_com,Roadmap to disaster? | Israeli gov't wants to ...
2020-05-12 16:00:01,1260238309445349379,Greenbackd,"""This is how I reason from first principles to..."
2020-05-12 16:00:01,1260238310137401344,SputnikInt,Africa’s richest woman cries foul as 'fake pas...
2020-05-12 16:00:02,1260238310728855552,WSJ,Meredith Vieira wasn’t interested in doing tal...
2020-05-12 16:00:02,1260238314180927489,DailyFXTeam,Fed's Quarles: - Jobless rate to be very high...


In [4]:
tweet_df = tweet_df[tweet_df.index < '06-01-2020']

In [5]:
single_day = tweet_df['2016-11-09']

In [3]:
ts =TopicSeries()
ts.calculate_day('2016-11-09', tweet_df['2016-11-09'].tweet)
ts.calculate_day('2016-05-08', tweet_df['2016-05-08'].tweet)

In [25]:
ts.models['2016-11-09']

[LatentDirichletAllocation(n_components=5, random_state=42),
 CountVectorizer(tokenizer=<function TopicSeries.twitter_tokenizer at 0x7fc5ab2cb320>),
 NMF(n_components=5, random_state=5),
 TfidfVectorizer(tokenizer=<function TopicSeries.twitter_tokenizer at 0x7fc5ab2cb320>)]

In [10]:
def display_components(model, word_features, top_display=5):
    # utility for displaying respresentative words per component for topic models
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        top_words_idx = topic.argsort()[::-1][:top_display]
        top_words = [word_features[i] for i in top_words_idx]
        print(" ".join(top_words))

In [11]:
sub = [lda, cv, nmf, tfidf]

In [4]:
import pickle
pickle.dump(ts, open( "save.p", "wb" ))

In [5]:
import pickle
a = pickle.load(open( "save.p", "rb" ))

In [8]:
a.models['2016-11-09']

[LatentDirichletAllocation(n_components=5, random_state=42),
 CountVectorizer(tokenizer=<function TopicSeries.twitter_tokenizer at 0x7ffe7bc88440>),
 NMF(n_components=5, random_state=5),
 TfidfVectorizer(tokenizer=<function TopicSeries.twitter_tokenizer at 0x7ffe7bc88440>)]

In [12]:
display_components(a.models['2016-05-08'][2], a.models['2016-05-08'][3].get_feature_names(), top_display=20)

Topic 0:
day happy mother mothers mom moms love #mothersday great thank wife nice time celebrating appreciation important today life enjoy world
Topic 1:
trump donald says clinton hillary gop party women republican ryan going president win taxes paul nominee wealthy better candidate said
Topic 2:
lol yep seen look actual idiot irl pic pro true picks trying fireworks nails choice texas nice huh answer haha
Topic 3:
good luck morning point love story man pretty night new nt book true time years listen wo saudi change add
Topic 4:
like know thanks think people yes new look right sure going time saudi oh great actually minister china oil looks
